In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from nltk.corpus import stopwords
import itertools
import nltk
import re
tqdm.pandas()
import ast

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

nltk.download('stopwords')

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /cluster/home/yakram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
LANGUAGE = 'de'
DATA_DIR = 'german'
stop_words = stopwords.words(DATA_DIR)

In [3]:
SPARQL_DIR = f'/cluster/scratch/yakram/sbert-copy/turtle_files/{DATA_DIR}_data/'

concept_schemes = pd.read_csv(os.path.join(SPARQL_DIR, 'query_concept_schemes_combined.rq'), sep='\t',
                              index_col=[0])

negatives_file = pd.read_csv('/cluster/scratch/yakram/sbert-copy/turtle_files/negative_100.csv', sep='\t',
                            index_col=[0])

MemConcept_prefLabel = pd.read_csv(os.path.join(SPARQL_DIR, 'query_MemConcept_prefLabel.rq'), sep='\t', index_col=[0])
SkillsHier_prefLabel = pd.read_csv(os.path.join(SPARQL_DIR, 'query_SkillsHier_prefLabel.rq'), sep='\t', index_col=[0])

MemConcept_altLabel = pd.read_csv(os.path.join(SPARQL_DIR, 'query_MemConcept_altLabel.rq'), sep='\t', index_col=[0])
SkillsHier_altLabel = pd.read_csv(os.path.join(SPARQL_DIR, 'query_SkillsHier_altLabel.rq'), sep='\t', index_col=[0])


MemConcept_hiddenLabel = pd.read_csv(os.path.join(SPARQL_DIR, 'query_MemConcept_hiddenLabel.rq'), sep='\t', index_col=[0])
SkillsHier_hiddenLabel = pd.read_csv(os.path.join(SPARQL_DIR, 'query_SkillsHier_hiddenLabel.rq'), sep='\t', index_col=[0])

MemConcept_description = pd.read_csv(os.path.join(SPARQL_DIR, 'query_MemConcept_description.rq'), sep='\t', index_col=[0])
SkillsHier_description = pd.read_csv(os.path.join(SPARQL_DIR, 'query_SkillsHier_description.rq'), sep='\t', index_col=[0])

memconcept_skills_prefLabel_combined = pd.concat([MemConcept_prefLabel, SkillsHier_prefLabel])
memconcept_skills_altLabel_combined = pd.concat([MemConcept_altLabel, SkillsHier_altLabel])
memconcept_skills_hiddenLabel_combined = pd.concat([MemConcept_hiddenLabel, SkillsHier_hiddenLabel])
memconcept_skills_description_combined = pd.concat([MemConcept_description, SkillsHier_description])


##### useful datasets #####


memconcept_skills_prefLabel = concept_schemes.merge(memconcept_skills_prefLabel_combined, on=['?a'], how='left')
memconcept_skills_altLabel = concept_schemes.merge(memconcept_skills_altLabel_combined, on=['?a'], how='left')
memconcept_skills_hiddenLabel = concept_schemes.merge(memconcept_skills_hiddenLabel_combined, on=['?a'], how='left')
memconcept_skills_description = concept_schemes.merge(memconcept_skills_description_combined, on=['?a'], how='left')

In [4]:
del MemConcept_prefLabel
del SkillsHier_prefLabel
del MemConcept_altLabel
del SkillsHier_altLabel
del MemConcept_hiddenLabel
del SkillsHier_hiddenLabel
del MemConcept_description
del SkillsHier_description
del memconcept_skills_prefLabel_combined
del memconcept_skills_altLabel_combined
del memconcept_skills_hiddenLabel_combined
del memconcept_skills_description_combined

In [5]:
negatives_file.head()

,main_uri,negative_uris
0,http://data.europa.eu/esco/isced-f/00,['http://data.europa.eu/esco/skill/b74fd765-e1...
1,http://data.europa.eu/esco/isced-f/000,['http://data.europa.eu/esco/skill/b779205e-37...
2,http://data.europa.eu/esco/isced-f/0000,['http://data.europa.eu/esco/skill/1fe4c04b-30...
3,http://data.europa.eu/esco/isced-f/001,['http://data.europa.eu/esco/skill/22d363fa-45...
4,http://data.europa.eu/esco/isced-f/0011,['http://data.europa.eu/esco/skill/aef7eab0-ae...


In [6]:
negatives_file.columns = ['?a', 'negative_uris']

In [7]:
neg_prefLabel_merged = memconcept_skills_prefLabel.merge(negatives_file, on=['?a'], how='left')
neg_altLabel_merged = memconcept_skills_altLabel.merge(negatives_file, on=['?a'], how='left')
neg_hiddenLabel_merged = memconcept_skills_hiddenLabel.merge(negatives_file, on=['?a'], how='left')
neg_description_merged = memconcept_skills_description.merge(negatives_file, on=['?a'], how='left')

neg_prefLabel_merged.dropna(inplace=True)
neg_altLabel_merged.dropna(inplace=True)
neg_hiddenLabel_merged.dropna(inplace=True)
neg_description_merged.dropna(inplace=True)

neg_prefLabel_merged['negative_uris'] = neg_prefLabel_merged['negative_uris'].apply(lambda x: ast.literal_eval(x))
neg_altLabel_merged['negative_uris'] = neg_altLabel_merged['negative_uris'].apply(lambda x: ast.literal_eval(x))
neg_hiddenLabel_merged['negative_uris'] = neg_hiddenLabel_merged['negative_uris'].apply(lambda x: ast.literal_eval(x))
neg_description_merged['negative_uris'] = neg_description_merged['negative_uris'].apply(lambda x: ast.literal_eval(x))

neg_prefLabel = neg_prefLabel_merged.explode('negative_uris')
neg_altLabel = neg_altLabel_merged.explode('negative_uris')
neg_hiddenLabel = neg_hiddenLabel_merged.explode('negative_uris')
neg_description = neg_description_merged.explode('negative_uris')

In [8]:
neg_prefLabel.rename(columns={neg_prefLabel.columns[1]: 'anchor'}, inplace=True)
neg_altLabel.rename(columns={neg_altLabel.columns[1]: 'anchor'}, inplace=True)
neg_hiddenLabel.rename(columns={neg_hiddenLabel.columns[1]: 'anchor'}, inplace=True)
neg_description.rename(columns={neg_description.columns[1]: 'anchor'}, inplace=True)

In [9]:
neg_prefLabel.rename(columns={ '?a': 'positive_uri','negative_uris': '?a'}, inplace=True)
neg_altLabel.rename(columns={ '?a': 'positive_uri','negative_uris': '?a'}, inplace=True)
neg_hiddenLabel.rename(columns={ '?a': 'positive_uri','negative_uris': '?a'}, inplace=True)
neg_description.rename(columns={ '?a': 'positive_uri','negative_uris': '?a'}, inplace=True)

In [10]:
neg_prefLabel.head()

,positive_uri,anchor,?a
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/815d1585-1706...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/5e433def-210d...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/5d1a7c14-188d...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/8b6f2996-218b...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,Lebensmittelrohstoffe lagern,http://data.europa.eu/esco/skill/779f5db1-66db...


In [11]:
neg_altLabel.head()

,positive_uri,anchor,?a
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,rohe Lebensmittel lagern,http://data.europa.eu/esco/skill/815d1585-1706...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,rohe Lebensmittel lagern,http://data.europa.eu/esco/skill/5e433def-210d...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,rohe Lebensmittel lagern,http://data.europa.eu/esco/skill/5d1a7c14-188d...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,rohe Lebensmittel lagern,http://data.europa.eu/esco/skill/8b6f2996-218b...
0,http://data.europa.eu/esco/skill/29b326ea-a9c8...,rohe Lebensmittel lagern,http://data.europa.eu/esco/skill/779f5db1-66db...


### attach negative term and get intersections

In [12]:
neg_pref_pos_altLabel = neg_prefLabel.merge(memconcept_skills_altLabel, on=['?a'], how='left')
neg_pref_pos_hiddenLabel = neg_prefLabel.merge(memconcept_skills_hiddenLabel, on=['?a'], how='left')
neg_pref_pos_description = neg_prefLabel.merge(memconcept_skills_description, on=['?a'], how='left')
neg_alt_pos_hidden = neg_altLabel.merge(memconcept_skills_hiddenLabel, on=['?a'], how='left')

In [13]:
neg_pref_pos_altLabel.rename(columns={neg_pref_pos_altLabel.columns[-1]: 'negative_term'}, inplace=True)
neg_pref_pos_hiddenLabel.rename(columns={neg_pref_pos_hiddenLabel.columns[-1]: 'negative_term'}, inplace=True)
neg_pref_pos_description.rename(columns={neg_pref_pos_description.columns[-1]: 'negative_term'}, inplace=True)
neg_alt_pos_hidden.rename(columns={neg_alt_pos_hidden.columns[-1]: 'negative_term'}, inplace=True)

In [14]:
neg_pref_pos_altLabel.dropna(inplace=True)
neg_pref_pos_hiddenLabel.dropna(inplace=True)
neg_pref_pos_description.dropna(inplace=True)
neg_alt_pos_hidden.dropna(inplace=True)

In [15]:
neg_pref_pos_altLabel.drop_duplicates(inplace=True)
neg_pref_pos_hiddenLabel.drop_duplicates(inplace=True)
neg_pref_pos_description.drop_duplicates(inplace=True)
neg_alt_pos_hidden.drop_duplicates(inplace=True)

In [16]:
neg_pref_pos_altLabel.shape,neg_pref_pos_hiddenLabel.shape,neg_pref_pos_description.shape,neg_alt_pos_hidden.shape,

((513329, 4), (56145, 4), (1392164, 4), (19488, 4))

In [17]:
def check_subset(term1, term2):
    term1 = term1.lower()
    term2 = term2.lower()
    
    clean_term1 = re.sub(r'\W+', ' ',term1).split() # for german, only \W+
    clean_term2 = re.sub(r'\W+', ' ', term2).split()
    
    term1s_sw_removed = [term1_term for term1_term in clean_term1 if term1_term not in stop_words]
    desc_sw_removed = [desc_term for desc_term in clean_term2 if desc_term not in stop_words]
    
    if len(desc_sw_removed) > 0:
        intersections = len(set(term1s_sw_removed).intersection(desc_sw_removed))/len(desc_sw_removed) # remove stopwords, and also set a threshold for intersection before removal
    
        return intersections
    return 0

In [18]:
neg_pref_pos_altLabel['anchor_negative'] = neg_pref_pos_altLabel.parallel_apply(lambda x: check_subset(x.anchor, x.negative_term) , axis=1)
neg_pref_pos_hiddenLabel['anchor_negative'] = neg_pref_pos_hiddenLabel.parallel_apply(lambda x: check_subset(x.anchor, x.negative_term) , axis=1)
neg_pref_pos_description['anchor_negative'] = neg_pref_pos_description.parallel_apply(lambda x: check_subset(x.anchor, x.negative_term) , axis=1)
neg_alt_pos_hidden['anchor_negative'] = neg_alt_pos_hidden.parallel_apply(lambda x: check_subset(x.anchor, x.negative_term) , axis=1)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
neg_pref_pos_altLabel_intersected = neg_pref_pos_altLabel[neg_pref_pos_altLabel['anchor_negative'] > 0]
neg_pref_pos_hiddenLabel_intersected = neg_pref_pos_hiddenLabel[neg_pref_pos_hiddenLabel['anchor_negative'] > 0]
neg_pref_pos_description_intersected = neg_pref_pos_description[neg_pref_pos_description['anchor_negative'] > 0]
neg_alt_pos_hidden_intersected = neg_alt_pos_hidden[neg_alt_pos_hidden['anchor_negative'] > 0]

In [21]:
neg_pref_pos_altLabel_intersected.drop(columns=['?a'], inplace=True)
neg_pref_pos_hiddenLabel_intersected.drop(columns=['?a'], inplace=True)
neg_pref_pos_description_intersected.drop(columns=['?a'], inplace=True)
neg_alt_pos_hidden_intersected.drop(columns=['?a'], inplace=True)

KeyError: "['?a'] not found in axis"

In [24]:
neg_pref_pos_altLabel_intersected.rename(columns={'positive_uri': '?a'}, inplace=True)
neg_pref_pos_hiddenLabel_intersected.rename(columns={'positive_uri': '?a'}, inplace=True)
neg_pref_pos_description_intersected.rename(columns={'positive_uri': '?a'}, inplace=True)
neg_alt_pos_hidden_intersected.rename(columns={'positive_uri': '?a'}, inplace=True)

/scratch/tmp.7590252.yakram/ipykernel_50797/2426843030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_pref_pos_altLabel_intersected.rename(columns={'positive_uri': '?a'}, inplace=True)
/scratch/tmp.7590252.yakram/ipykernel_50797/2426843030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_pref_pos_hiddenLabel_intersected.rename(columns={'positive_uri': '?a'}, inplace=True)
/scratch/tmp.7590252.yakram/ipykernel_50797/2426843030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [25]:
neg_pref_pos_altLabel_intersected.head()

,?a,anchor,negative_term,anchor_negative
386,http://data.europa.eu/esco/skill/7e11b04d-c6b5...,Patienten/Patientinnen zur Verbesserung des Sp...,Patienten/Patientin an Ophthalmologen/Ophthalm...,0.200000
1303,http://data.europa.eu/esco/skill/3838a64c-78dc...,Systeme zur internen Kommunikation warten,landwirtschaftliches Gerät warten,0.333333
2056,http://data.europa.eu/esco/skill/83f713fb-2650...,Signalgeräte für Züge bedienen,Nagelungsmaschinen bedienen,0.500000
3698,http://data.europa.eu/esco/skill/c349b18e-5038...,Foliendruckmaschine bedienen,Wiegeanlage bedienen,0.500000
3742,http://data.europa.eu/esco/skill/c349b18e-5038...,Foliendruckmaschine bedienen,Kuvertmaschine bedienen,0.500000


In [26]:
neg_pref_pos_hiddenLabel_intersected.head()

,?a,anchor,negative_term,anchor_negative
59511,http://data.europa.eu/esco/skill/1bc718b6-ed90...,Creative-Suite-Software benutzen,Empire Suite Software,0.666667
144075,http://data.europa.eu/esco/skill/ae57a9c3-f761...,Software für die Verkaufsprognose nutzen,Empire Suite Software,0.333333
144342,http://data.europa.eu/esco/skill/25fce4fb-35b6...,Ware untersuchen,implement SPAM-ware,0.333333
707369,http://data.europa.eu/esco/skill/56a10e3e-c9e7...,Software für Content-Management-Systeme nutzen,Empire Suite Software,0.333333
1416961,http://data.europa.eu/esco/skill/5303169c-75d6...,Social Media Management,Nuxeo Document Management,0.333333


In [27]:
neg_pref_pos_altLabel_intersected_positive_term =neg_pref_pos_altLabel_intersected.merge(memconcept_skills_altLabel, on=['?a'], how='left')
neg_pref_pos_hiddenLabel_intersected_positive_term =neg_pref_pos_hiddenLabel_intersected.merge(memconcept_skills_hiddenLabel, on=['?a'], how='left')
neg_pref_pos_description_intersected_positive_term =neg_pref_pos_description_intersected.merge(memconcept_skills_description, on=['?a'], how='left')
neg_alt_pos_hidden_intersected_positive_term =neg_alt_pos_hidden_intersected.merge(memconcept_skills_hiddenLabel, on=['?a'], how='left')

In [28]:
neg_pref_pos_altLabel_intersected_positive_term.dropna(inplace=True)
neg_pref_pos_hiddenLabel_intersected_positive_term.dropna(inplace=True)
neg_pref_pos_description_intersected_positive_term.dropna(inplace=True)
neg_alt_pos_hidden_intersected_positive_term.dropna(inplace=True)

In [29]:
neg_pref_pos_altLabel_intersected_positive_term.rename(columns={neg_pref_pos_altLabel_intersected_positive_term.columns[-1]: 'positive_term'}, inplace=True)
neg_pref_pos_hiddenLabel_intersected_positive_term.rename(columns={neg_pref_pos_hiddenLabel_intersected_positive_term.columns[-1]: 'positive_term'}, inplace=True)
neg_pref_pos_description_intersected_positive_term.rename(columns={neg_pref_pos_description_intersected_positive_term.columns[-1]: 'positive_term'}, inplace=True)
neg_alt_pos_hidden_intersected_positive_term.rename(columns={neg_alt_pos_hidden_intersected_positive_term.columns[-1]: 'positive_term'}, inplace=True)

In [33]:
if LANGUAGE == 'en':
    triplet_pref_altLabel = neg_pref_pos_altLabel_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()
    triplet_pref_hiddenLabel = neg_pref_pos_hiddenLabel_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()
    triplet_pref_description = neg_pref_pos_description_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()
    triplet_alt_hidden = neg_alt_pos_hidden_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()
elif LANGUAGE == 'de':
    triplet_pref_altLabel = neg_pref_pos_altLabel_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()
    triplet_pref_description = neg_pref_pos_description_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()
    triplet_pref_hiddenLabel = pd.DataFrame(columns=triplet_pref_altLabel.columns)
    triplet_alt_hidden = pd.DataFrame(columns=triplet_pref_altLabel.columns)

/scratch/tmp.7590252.yakram/ipykernel_50797/182013951.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  triplet_pref_altLabel = neg_pref_pos_altLabel_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()
/scratch/tmp.7590252.yakram/ipykernel_50797/182013951.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  triplet_pref_description = neg_pref_pos_description_intersected_positive_term.groupby(by=['?a', 'anchor', 'positive_term'])['?a', 'anchor', 'positive_term', 'negative_term'].sample(1).reset_index()


In [34]:
triplet_pref_altLabel.drop(columns=['index'], inplace=True)
triplet_pref_hiddenLabel.drop(columns=['index'], inplace=True)
triplet_pref_description.drop(columns=['index'], inplace=True)
triplet_alt_hidden.drop(columns=['index'], inplace=True)

In [35]:
neg_pref_pos_altLabel_intersected_positive_term.drop(columns=['anchor_negative'], inplace=True)
neg_pref_pos_hiddenLabel_intersected_positive_term.drop(columns=['anchor_negative'], inplace=True)
neg_pref_pos_description_intersected_positive_term.drop(columns=['anchor_negative'], inplace=True)
neg_alt_pos_hidden_intersected_positive_term.drop(columns=['anchor_negative'], inplace=True)

In [36]:
neg_pref_pos_altLabel_intersected_positive_term.head()

,?a,anchor,negative_term,positive_term
19,http://data.europa.eu/esco/skill/c747ab82-9c35...,den Vorbereitungsbereich reinigen,Maschinen von Abfallstoffen reinigen,den Küchenbereich reinigen
20,http://data.europa.eu/esco/skill/c747ab82-9c35...,den Vorbereitungsbereich reinigen,Maschinen von Abfallstoffen reinigen,die Küche reinigen
21,http://data.europa.eu/esco/skill/c747ab82-9c35...,den Vorbereitungsbereich reinigen,Maschinen von Abfallstoffen reinigen,den Zubereitungsbereich saubermachen
40,http://data.europa.eu/esco/skill/d89f5cee-649a...,die Leistung von Vertragspartnern überwachen,Tunnelbohrmaschinen-Betriebsmittel überwachen,Leistung der Vertragspartner überwachen
41,http://data.europa.eu/esco/skill/d89f5cee-649a...,die Leistung von Vertragspartnern überwachen,Tunnelbohrmaschinen-Betriebsmittel überwachen,die Leistung von Vertragspartnern beaufsichtigen


In [37]:
triplet_pref_altLabel.head()

,?a,anchor,positive_term,negative_term
0,http://data.europa.eu/esco/skill/0058526a-11e9...,Pflanzenforschung durchführen,Forschung zur Flora durchführen,Systemprotokollierung durchführen
1,http://data.europa.eu/esco/skill/03353964-657a...,Tischgedecke prüfen,Anordnung des Geschirrs und Bestecks prüfen,Studioproduktion prüfen
2,http://data.europa.eu/esco/skill/03353964-657a...,Tischgedecke prüfen,Sicherstellen dass der Tisch richtig gedeckt ist,Studioproduktion prüfen
3,http://data.europa.eu/esco/skill/03353964-657a...,Tischgedecke prüfen,die Anordnung des Tischgedecks prüfen,Studioproduktion prüfen
4,http://data.europa.eu/esco/skill/03a0ab28-ce99...,Eventmanagement durchführen,Veranstaltungsmanagement durchführen,Konditionierung von Tabakblättern durchführen


### dev set mining

In [38]:
triplet_pref_altLabel_values = triplet_pref_altLabel['negative_term'].unique().tolist()
triplet_pref_hiddenLabel_values = triplet_pref_hiddenLabel['negative_term'].unique().tolist()
triplet_pref_description_values = triplet_pref_description['negative_term'].unique().tolist()
triplet_alt_hidden_values = triplet_alt_hidden['negative_term'].unique().tolist()

In [39]:
len(triplet_pref_altLabel_values),len(triplet_pref_hiddenLabel_values),len(triplet_pref_description_values),len(triplet_alt_hidden_values),

(279, 0, 4586, 0)

In [40]:
full_values_list = list(itertools.chain(triplet_pref_altLabel_values,triplet_pref_hiddenLabel_values,triplet_pref_description_values,triplet_alt_hidden_values))

In [41]:
def flag_intersected_term(negative_column, covered_list):
    
    if negative_column in covered_list:
        return 'present'
    return 'absent'

In [42]:
neg_pref_pos_altLabel_intersected_positive_term['already_covered'] = neg_pref_pos_altLabel_intersected_positive_term.parallel_apply(
lambda x: flag_intersected_term(x.negative_term, full_values_list), axis=1)

In [44]:
if LANGUAGE == 'en':
    neg_pref_pos_hiddenLabel_intersected_positive_term['already_covered'] = neg_pref_pos_hiddenLabel_intersected_positive_term.parallel_apply(
lambda x: flag_intersected_term(x.negative_term, full_values_list), axis=1)
elif LANGUAGE == 'de':
    neg_pref_pos_hiddenLabel_intersected_positive_term = pd.DataFrame(columns=neg_pref_pos_altLabel_intersected_positive_term.columns)

In [45]:
neg_pref_pos_description_intersected_positive_term['already_covered'] = neg_pref_pos_description_intersected_positive_term.parallel_apply(
lambda x: flag_intersected_term(x.negative_term, full_values_list), axis=1)

In [46]:
if LANGUAGE == 'en':
    neg_alt_pos_hidden_intersected_positive_term['already_covered'] = neg_alt_pos_hidden_intersected_positive_term.parallel_apply(
lambda x: flag_intersected_term(x.negative_term, full_values_list), axis=1)
elif LANGUAGE == 'de':
    neg_alt_pos_hidden_intersected_positive_term = pd.DataFrame(columns=neg_pref_pos_altLabel_intersected_positive_term.columns)

In [48]:
dev_neg_pref_pos_altLabel_intersected_positive_term = neg_pref_pos_altLabel_intersected_positive_term[neg_pref_pos_altLabel_intersected_positive_term['already_covered'] == 'absent']
dev_neg_pref_pos_hiddenLabel_intersected_positive_term = neg_pref_pos_hiddenLabel_intersected_positive_term[neg_pref_pos_hiddenLabel_intersected_positive_term['already_covered'] == 'absent']
dev_neg_pref_pos_description_intersected_positive_term = neg_pref_pos_description_intersected_positive_term[neg_pref_pos_description_intersected_positive_term['already_covered'] == 'absent']
dev_neg_alt_pos_hidden_intersected_positive_term = neg_alt_pos_hidden_intersected_positive_term[neg_alt_pos_hidden_intersected_positive_term['already_covered'] == 'absent']

In [49]:
dev_neg_pref_pos_altLabel_intersected_positive_term.shape,dev_neg_pref_pos_hiddenLabel_intersected_positive_term.shape,dev_neg_pref_pos_description_intersected_positive_term.shape,dev_neg_alt_pos_hidden_intersected_positive_term.shape,

((137, 5), (0, 5), (3976, 5), (0, 5))

In [50]:
dev_sets = [dev_neg_pref_pos_altLabel_intersected_positive_term,dev_neg_pref_pos_hiddenLabel_intersected_positive_term,dev_neg_pref_pos_description_intersected_positive_term,dev_neg_alt_pos_hidden_intersected_positive_term]

In [52]:
all_dev_set_data = pd.concat(dev_sets)

In [53]:
all_dev_set_data.shape

(4113, 5)

In [55]:
if LANGUAGE == 'en':
    dev_dataset = all_dev_set_data.sample(n=20000)
else:
    dev_dataset = all_dev_set_data

In [56]:
dev_dataset.drop(columns=['already_covered'], inplace=True)

### merging all triplets

In [57]:
triplets_combined = [triplet_pref_altLabel,triplet_pref_hiddenLabel,triplet_pref_description,triplet_alt_hidden]

In [58]:
mnr_data = pd.concat(triplets_combined)

In [59]:
mnr_data.dropna(inplace=True)

In [60]:
triplet_save_dir = f'/cluster/scratch/yakram/sbert-copy/turtle_files/mnr_data/{LANGUAGE}/hard_neg_triplets.csv'
triplet_save_dir

'/cluster/scratch/yakram/sbert-copy/turtle_files/mnr_data/de/hard_neg_triplets.csv'

In [63]:
mnr_data.shape

(7187, 4)

In [64]:
mnr_data.to_csv(triplet_save_dir, sep='\t')

In [65]:
dev_dataset_dir = f'/cluster/scratch/yakram/sbert-copy/turtle_files/mnr_data/{LANGUAGE}/hard_neg_dev.csv'
dev_dataset_dir

'/cluster/scratch/yakram/sbert-copy/turtle_files/mnr_data/de/hard_neg_dev.csv'

In [66]:
dev_dataset.shape

(4113, 4)

In [67]:
dev_dataset.to_csv(dev_dataset_dir, sep='\t')